# Abstract

Floods are among the most devastating natural disasters, causing significant economic, social, and environmental damage. 
Timely and accurate flood detection is crucial for disaster response and mitigation efforts. 
This project presents a flood detection system based on Sentinel-1 Synthetic Aperture Radar (SAR) data, validated against a water mask derived from Sentinel-2 optical imagery using the Normalized Difference Water Index (NDWI).

The methodology involved the preprocessing of SAR and optical data, calculation of NDWI, generation of flood masks, and validation through a confusion matrix analysis.
Results showed that the SAR-based flood mask achieved a precision of 100%, indicating a high reliability in positively identified flood areas, but a recall of 6.39%, highlighting missed flood detections.

Limitations included the absence of validation data and the influence of cloud cover on optical imagery. 
Despite these challenges, the project demonstrated the potential of remote sensing technologies for flood detection and provides a foundation for future work.


# Setting Up the Environment

Importing Libraries & Starting Earth Engine
In this phase, we import necessary libraries and initialize Google Earth Engine to fetch and process satellite images for a specific area and time range.

In [ ]:
import ee
import geemap
import os
import requests
import rasterio
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd




# Initialize the Earth Engine API 
ee.Authenticate()
ee.Initialize(project='atu-fyp')

## Choosing the area and time range

We define the AOI for Cork, Ireland, which was heavily affected by floods in 2023. This AOI will be used to fetch and analyze satellite images.

In [ ]:
# Define the Area of Interest (AOI)
def create_aoi(lat_north, lat_south, lon_west, lon_east, buffer_size=1000):
    """
    Creates an AOI polygon with a buffer.
    Args:
        lat_north (float): Northern latitude.
        lat_south (float): Southern latitude.
        lon_west (float): Western longitude.
        lon_east (float): Eastern longitude.
        buffer_size (int): Buffer size in meters.
    Returns:
        ee.Geometry: Buffered AOI polygon.
    """
    return ee.Geometry.Polygon([
        [
            [lon_west, lat_north],
            [lon_west, lat_south],
            [lon_east, lat_south],
            [lon_east, lat_north]
        ]
    ]).buffer(buffer_size)


aoi = create_aoi(51.95, 51.80, -8.60, -8.15)

# Initialize the map
Map = geemap.Map(center=[51.8691, -8.2646], zoom=10)
Map.addLayer(aoi, {'color': 'brown'}, 'Area of Interest')
Map

## Loading Sentinel-1 SAR Images

This function loads Sentinel-1 radar images for a given date range and area.  
We use it to get a clean image before and after the flood for comparison.


In [ ]:
# Define date ranges for pre-flood and post-flood periods
def define_date_ranges(pre_start, pre_end, post_start, post_end):
    """
    Defines date ranges for pre-flood and post-flood periods.
    Args:
        pre_start (str): Start date for pre-flood period (YYYY-MM-DD).
        pre_end (str): End date for pre-flood period (YYYY-MM-DD).
        post_start (str): Start date for post-flood period (YYYY-MM-DD).
        post_end (str): End date for post-flood period (YYYY-MM-DD).
    Returns:
        dict: Dictionary containing the date ranges.
    """
    return {
        'pre_flood_start': pre_start,
        'pre_flood_end': pre_end,
        'post_flood_start': post_start,
        'post_flood_end': post_end
    }


date_ranges = define_date_ranges('2023-09-01', '2023-09-30', '2023-10-01', '2023-10-31')
pre_flood_start = date_ranges['pre_flood_start']
pre_flood_end = date_ranges['pre_flood_end']
post_flood_start = date_ranges['post_flood_start']
post_flood_end = date_ranges['post_flood_end']

# Data collection Loading and Visualization


## 4.1 Loading Sentinel-1 Data
This section loads and filters Sentinel-1 satellite radar data for the specified flood event timeframes.  
It prepares the data for analysis by applying filters such as date, area, and polarization.

In [ ]:
# Load Sentinel-1 data
def load_sentinel1_data(start_date, end_date, aoi):
    """
    Loads Sentinel-1 GRD data for a given date range and AOI.
    Args:
        start_date (str): Start date (YYYY-MM-DD).
        end_date (str): End date (YYYY-MM-DD).
        aoi (ee.Geometry): Area of Interest.
    Returns:
        ee.Image: Median composite of Sentinel-1 data.
    """
    return ee.ImageCollection('COPERNICUS/S1_GRD') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
        .filter(ee.Filter.eq('instrumentMode', 'IW')) \
        .select('VV').median()

pre_flood_image = load_sentinel1_data(pre_flood_start, pre_flood_end, aoi)
post_flood_image = load_sentinel1_data(post_flood_start, post_flood_end, aoi)

In [ ]:
# Here we add the pre-flood and post-flood Sentinel-1 radar images to the map.  
pre_flood_image = load_sentinel1_data(pre_flood_start, pre_flood_end, aoi)
post_flood_image = load_sentinel1_data(post_flood_start, post_flood_end, aoi)

# Add pre-flood and post-flood images to the map
Map.addLayer(pre_flood_image.clip(aoi), {'min': -20, 'max': 0}, 'Pre-Flood VV')
Map.addLayer(post_flood_image.clip(aoi), {'min': -20, 'max': 0}, 'Post-Flood VV')
Map.centerObject(aoi)
Map

## Exporting Pre- and Post-Flood Images

In [ ]:
# Define the export folder and filenames
export_folder = r'C:\Users\Neo\Desktop\FYP\Flood-Detection-System-FYP\Gee\images\flood_images'
pre_flood_filename = 'Pre_Flood_VV.png'
post_flood_filename = 'Post_Flood_VV.png'

# Ensure the export folder exists
os.makedirs(export_folder, exist_ok=True)

# Function to download thumbnails directly from Earth Engine to a local folder.
def download_image_as_png(image, filename, vis_params=None):
    try:
        # Generate the thumbnail URL
        url = image.getThumbURL({
            'dimensions': 1024,  # Set the dimensions of the PNG
            'region': aoi.getInfo()['coordinates'],  # Export region
            'format': 'png',  # Save as PNG
            'min': -20,  # Minimum value for visualization
            'max': 0,  # Maximum value for visualization
            'palette': ['black', 'white']  # Grayscale palette
        })
        # Download the image
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            filepath = os.path.join(export_folder, filename)
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    f.write(chunk)
            print(f"Saved: {filepath}")
        else:
            print(f"Failed to download {filename}. HTTP Status Code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {filename}: {e}")

# Download Pre-Flood VV image as PNG
download_image_as_png(pre_flood_image, pre_flood_filename)

# Download Post-Flood VV image as PNG
download_image_as_png(post_flood_image, post_flood_filename)

## Water Area Time Series Analysis

Here we calculate the water extent for each Sentinel-1 image during the flood period.  
The water area is estimated by classifying pixels with low VV backscatter, and the results are plotted over time to show how flooding evolved.


In [ ]:
full_collection = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate('2023-09-01', '2023-10-31') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('instrumentMode', 'IW'))

# Function to calculate water extent for each image
def calc_water_extent(image):
    water = image.select('VV').lt(-15)  # Pixels with VV backscatter below -15 dB are classified as floodwater.
    area = water.multiply(ee.Image.pixelArea())
    stats = area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=aoi,
        scale=30
    )
    return image.set({'water_area': stats.get('VV'), 'date': image.date().format()})

# Apply function to image collection
time_series = full_collection.map(calc_water_extent)

# Export results for plotting
timeseries_stats_m2 = time_series.aggregate_array('water_area').getInfo()
dates = time_series.aggregate_array('date').getInfo()

# Convert area from m² to km²
timeseries_stats_km2 = [value / 1e6 if value else 0 for value in timeseries_stats_m2]

# Plotting
import matplotlib.pyplot as plt
plt.figure(figsize=(12, 6))
plt.plot(dates, timeseries_stats_km2, marker='o')
plt.title('Water Area Time Series')
plt.ylabel('Area (km²)')
plt.xticks(rotation=45)
plt.grid()
plt.savefig('images/charts/Water_Area_Time_Series_km2.png', dpi=300, bbox_inches='tight')
plt.show()


## Analyzing Flood Related Changes
This section calculates the difference between pre-flood and post-flood images to detect changes in water-covered areas. A threshold is applied to identify flooded areas.

## Exporting the Difference Image

We calculate the difference between the post-flood and pre-flood radar images to highlight flood changes.  
The difference image is saved as a PNG file with a color palette showing where significant changes happened.


In [ ]:
# Define the export folder and filename
export_folder = r'C:\Users\Neo\Desktop\FYP\Flood-Detection-System-FYP\Gee\images\flood_images'
filename = 'Difference_VV.png'

# Ensure the export folder exists
os.makedirs(export_folder, exist_ok=True)

# Calculate the difference between post-flood and pre-flood images
difference_image = post_flood_image.subtract(pre_flood_image)

# Debug: Print the difference image info
print("Difference Image Info:", difference_image.getInfo())

def download_difference_image_as_png(image, filename):
    try:
        # Generate the thumbnail URL
        url = image.getThumbURL({
            'dimensions': 1024,  # Set the dimensions of the PNG
            'region': aoi.getInfo()['coordinates'],  # Export region
            'format': 'png',  # Save as PNG
            'min': -5,  # Minimum value for visualization
            'max': 5,  # Maximum value for visualization
            'palette': ['red', 'white', 'blue']  # Color palette for difference
        })

        # Debug: Print the generated URL
        print(f"Generated URL: {url}")

        # Download the image
        response = requests.get(url, stream=True)
        # Debug: Print the HTTP status code
        print(f"HTTP Status Code: {response.status_code}")

        if response.status_code == 200:
            filepath = os.path.join(export_folder, filename)
            # Debug: Print the file path
            print(f"Saving to: {filepath}")

            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=1024):
                    f.write(chunk)
            print(f"Saved: {filepath}")
        else:
            print(f"Failed to download {filename}. HTTP Status Code: {response.status_code}")
    except Exception as e:
        print(f"Error downloading {filename}: {e}")

# Download the difference image
download_difference_image_as_png(difference_image, filename)

# Add the difference image to the map
Map.addLayer(difference_image.clip(aoi), {'min': -5, 'max': 5, 'palette': ['blue', 'white', 'red']}, 'Difference Image')
Map.centerObject(aoi)
Map

EXPLANATION

We use colors to show the changes. Red means a lot of change, blue means less change, and white means no change.
Flood Mask: We use blue to show the places where we think there was a flood. This helps us see the flooded areas on the map.

# Creating the Flood Mask

We apply a threshold to the difference image to detect flooded areas.  
Pixels above the threshold are classified as flooded and visualized on the map.


In [ ]:
# Define a threshold for flood detection
threshold = 2  # Adjust this value based on your analysis

# Create a binary flood mask
flood_mask = difference_image.gt(threshold)

# Add the flood mask to the map
Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'Flood Mask')
Map.centerObject(aoi)
Map

EXPLANATION

Thresholding: We decide how much change means there was a flood.  If the change is more than 2, there was a flood.

## Exporting the Flood Mask

The final flood mask is exported to Google Drive.  
The export runs as a background task and saves the result at 30-meter resolution.


In [ ]:
# Export the flood mask to Google Drive
task = ee.batch.Export.image.toDrive(
    image=flood_mask.clip(aoi),
    description='FloodMask',
    folder='EarthEngineImages',
    scale=30,
    region=aoi
)
task.start()

print("Export task started. Check Google Drive for the flood mask.")

## Loading and Displaying the Exported Flood Mask

We check if the exported flood mask file exists locally.  
If it does, we open it and display the flood areas using a simple blue color map.


In [ ]:

# Check if the file exists
import os
file_path = r"C:\\Users\\Neo\\Desktop\\FYP\\geeimages\\FloodMask.tif"
if os.path.exists(file_path):
    # Open the exported flood mask
    with rasterio.open(file_path) as src:
        flood_mask_image = src.read(1)

    # Display the flood mask
    plt.imshow(flood_mask_image, cmap='Blues', vmin=0, vmax=1)
    plt.colorbar(label='Flood Mask')
    plt.title('Flood Mask')
    plt.show()
else:
    print(f"File not found: {file_path}")

## Satellite Resolution Comparison

Here we compare Sentinel-2, Landsat-8, and Sentinel-1 based on their spatial and temporal resolution.  
This helps understand which satellite offers better detail and more frequent observations.


In [ ]:
satellites = ['Sentinel-2', 'Landsat-8', 'Sentinel-1']

# Spatial resolution in meters for each satellite (lower = better detail)
spatial_resolution = [10, 30, 10]

# Temporal resolution in days (how often the satellite revisits the same location)
temporal_resolution = [5, 16, 6]

# Create an index for the x-axis (one for each satellite)
x = range(len(satellites))

# Define the width of each bar
bar_width = 0.35

# Create a new figure and axis object
fig, ax = plt.subplots(figsize=(8, 5))  # Set figure size

# Plot the spatial resolution bars (shift left by half bar width)
ax.bar([i - bar_width/2 for i in x], spatial_resolution,
       width=bar_width, label='Spatial Resolution (m)')

# Plot the temporal resolution bars (shift right by half bar width)
ax.bar([i + bar_width/2 for i in x], temporal_resolution,
       width=bar_width, label='Temporal Resolution (days)')

# Add labels and title
ax.set_xlabel('Satellite')
ax.set_title('Comparison of Spatial and Temporal Resolution')
ax.set_xticks(x)  # Set the positions of the x-axis ticks
ax.set_xticklabels(satellites)  # Label the x-axis ticks with satellite names
ax.legend()  # Add a legend
ax.grid(True, linestyle='--', alpha=0.5)  # Optional: add light grid lines

# Adjust layout to avoid clipping
plt.tight_layout()

# Save the chart as an image file
plt.savefig('images/charts/satellite_comparison.png')  
plt.show()


## Visualizing Different Flood Detection Thresholds

We test multiple thresholds to see how they affect flood detection results.  
Each threshold generates a different flood mask layer, helping us choose the best value.


In [ ]:
# Defining a list of threshold values to test
thresholds = [1.5, 2, 2.5, 3]

# map to visualize the results
Map = geemap.Map(center=[51.8691, -8.2646], zoom=10)

# Loops through each threshold and add the flood mask to the map
for threshold in thresholds:
    flood_mask = difference_image.gt(threshold)
    Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, f'Flood Mask (Threshold={threshold})')

# Adds the AOI and difference image for reference
Map.addLayer(aoi, {'color': 'red'}, 'Area of Interest')
Map.addLayer(difference_image.clip(aoi), {'min': -5, 'max': 5, 'palette': ['red', 'white', 'blue']}, 'Difference Image')
Map.centerObject(aoi)
Map

EXPLANATION

Here wer are making sure we get it right

Flood Masks for Different Thresholds: Using blue to show the places where we think there was a flood for each number we tried. This helps see how each number works.

Visualizing the Results: See how well each number works. Addind the flood areas to the map for each number and see which one shows the floods most accurately.





# Calculating NDWI from Post-Flood Sentinel-2 Images

We load post-flood Sentinel-2 images, calculate the NDWI index to detect water bodies,  
and create a median composite. The NDWI layer is then added to the map for visualization.


In [ ]:
# Load Sentinel-2 images after the flood
post_flood_sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate(post_flood_start, post_flood_end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70))

# Function to calculate NDWI
def calculate_ndwi(image):
    """
    Calculates NDWI for a given Sentinel-2 image.
    Args:
        image (ee.Image): A Sentinel-2 image.
    Returns:
        ee.Image: The input image with an added 'NDWI' band.
    """
    return image.normalizedDifference(['B3', 'B8']).rename('NDWI')

# Apply NDWI calculation
ndwi_collection = post_flood_sentinel2.map(calculate_ndwi)

# Create a median composite NDWI image
ndwi_image = ndwi_collection.median()

# Visualization parameters
ndwi_vis_params = {
    'bands': ['NDWI'],
    'min': -0.3,
    'max': 0.3,
    'palette': ['white', 'blue']
}

# Add NDWI layer to the map
Map.addLayer(ndwi_image.clip(aoi), ndwi_vis_params, 'NDWI')
Map.centerObject(aoi)
Map


EXPLANATION

Sentinel-1 and Sentinel-2 We use two different satellites to get pictures. Sentinel-1 sees through clouds and Sentinel-2 sees colors.

NDWI Calculation We use Sentinel-2 to find water by looking at how much green and near-infrared light is reflected.

NDWI Water Mask We color the places where we think there is water using the NDWI calculation.

Comparing with Flood Mask We look at both the NDWI water mask and the flood mask from Sentinel-1 to see if they match.

We check if the places we found with Sentinel-1 match the places we found with Sentinel-2. This helps us make sure we are finding the floods correctly.

The NDWI layer is added to visualize and detect areas covered by water, enabling the identification of flood-affected zones by enhancing the contrast between water bodies and land in post-flood satellite pictures.


## Detecting Flooded Areas with NDWI Change

We calculate NDWI before and after the flood, then compare them to highlight flooded areas.  
Regions with a significant NDWI increase are classified as flooded and visualized on the map.


In [ ]:
# Function to standardize bands
def standardize_bands(image):
    return image.select(['B3', 'B8'])

# Function to add NDWI band
def add_ndwi(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)

# Load pre- and post-flood images
def load_sentinel2(start_date, end_date):
    return ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)) \
        .map(standardize_bands)

pre_flood = load_sentinel2(pre_flood_start, pre_flood_end).map(add_ndwi).median()
post_flood = load_sentinel2(post_flood_start, post_flood_end).map(add_ndwi).median()

# Print min/max NDWI values
print('Pre-flood NDWI min/max:', pre_flood.select('NDWI').reduceRegion(ee.Reducer.minMax(), aoi, 1000).getInfo())
print('Post-flood NDWI min/max:', post_flood.select('NDWI').reduceRegion(ee.Reducer.minMax(), aoi, 1000).getInfo())

# Visualization
ndwi_vis_params = {'bands': 'NDWI', 'min': -0.3, 'max': 0.3, 'palette': ['red', 'white', 'blue']}
Map.addLayer(pre_flood.clip(aoi), ndwi_vis_params, 'Pre-Flood NDWI')
Map.addLayer(post_flood.clip(aoi), ndwi_vis_params, 'Post-Flood NDWI')

# Flood detection
flood_diff = post_flood.select('NDWI').subtract(pre_flood.select('NDWI'))

# Mask areas with significant NDWI increase (>0.1) to highlight flooded areas
flooded_areas = flood_diff.gt(0.05).And(post_flood.select('NDWI').gt(0)).selfMask()
Map.addLayer(flooded_areas.clip(aoi), 
             {'palette': 'blue'}, 
             'Flooded Areas (NDWI Increase > 0.1)')

# Show the difference in NDWI
Map.addLayer(flood_diff.clip(aoi), {'min': 0, 'max': 0.5, 'palette': ['white', 'blue']}, 'NDWI Difference')
Map.addLayer(flood_diff.gt(0.1).selfMask().clip(aoi), {'palette': 'blue'}, 'Flooded Areas (NDWI Increase > 0.1)')
Map.centerObject(aoi)
Map


Explanation

| Step | What Happens? |
|:----|:--------------|
| **Pick bands** | Choose green and infrared light to detect water |
| **Add NDWI** | Create a "water index" to highlight water presence |
| **Load images** | Bring in pre-flood and post-flood satellite photos |
| **Filter clouds** | Use only clear satellite images |
| **Calculate difference** | Find where water coverage increased |
| **Highlight floods** | Show newly flooded areas in blue on the map |


## Interpretation of the Outputs

| **Layer**            | **Description**                                           |
|----------------------|-----------------------------------------------------------|
| Pre-Flood NDWI       | Red/white = land, blue = pre-existing water               |
| Post-Flood NDWI      | New blue areas = potential flooding                       |
| NDWI Difference      | White = no change, blue = increased water                 |
| Flooded Areas        | Only pixels where NDWI increased significantly            |

---

- If there was a flood, **Post-flood NDWI_max** should be **higher** than **Pre-flood NDWI_max**.
- **Water** usually gives **high positive NDWI** (closer to +1).
- **Dry land** has **low or negative NDWI** (around 0 or negative values).

The final flood detection (`flood_diff.gt(0.1)`) gives:
- A **masked image** where only pixels that increased NDWI by more than 0.1 are shown.
- These pixels are interpreted as **"new water" (flooded areas)**.

If there was **no flood**, this layer will look almost


## Loading and Masking Sentinel-2 Images

We load Sentinel-2 surface reflectance images and apply cloud masking to improve flood detection.  
Both raw and masked images are visualized to show the difference in quality.


In [ ]:
# Function to Load and Mask Sentinel-2 Data 
def load_and_mask(start_date, end_date):
    """Loads and masks clouds in Sentinel-2 SR data for a given date range"""
    collection = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(aoi) \
        .filterDate(start_date, end_date) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50))

    def mask_clouds(image):
        cloud_prob = image.select('MSK_CLDPRB')
        return image.updateMask(cloud_prob.lt(50))

    return collection.map(mask_clouds).median()

# Load cloud-masked images from the COPERNICUS/S2_SR dataset
pre_flood = load_and_mask(pre_flood_start, pre_flood_end)
post_flood = load_and_mask(post_flood_start, post_flood_end)

# Load Raw (Unmasked) Data for Comparison
raw_pre_flood = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(aoi) \
    .filterDate(pre_flood_start, pre_flood_end) \
    .median()

raw_post_flood = ee.ImageCollection('COPERNICUS/S2_SR') \
    .filterBounds(aoi) \
    .filterDate(post_flood_start, post_flood_end) \
    .median()

# Visualization Parameters
vis_params = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 10000,
    'gamma': 1.3
}

Map.addLayer(raw_post_flood.clip(aoi), vis_params, 'Raw Sentinel-2 Post-Flood')
Map.addLayer(post_flood.clip(aoi), vis_params, 'Masked Post-Flood')

# Add layer control
Map.addLayerControl()
Map


## Downloading Pre- and Post-Flood NDWI Images

We calculate NDWI for pre- and post-flood Sentinel-2 images and download them as PNG files.  
This allows us to keep local copies for reporting and analysis.


In [ ]:
import os
import urllib.request

# Define paths
dir = "images/flood_maps"
os.makedirs(dir, exist_ok=True)

# Correct NDWI calculation
def add_ndwi(image):
    return image.normalizedDifference(['B3', 'B8']).rename('NDWI')

ndwi_collection = post_flood_sentinel2.map(calculate_ndwi)
ndwi_image = ndwi_collection.median()


# Load pre- and post-flood images
pre_flood = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate(pre_flood_start, pre_flood_end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70)) \
    .map(add_ndwi) \
    .median()

post_flood = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate(post_flood_start, post_flood_end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 70)) \
    .map(add_ndwi) \
    .median()

# Function to save Earth Engine thumbnail
def save_ee_thumbnail(image, params, filename):
    url = image.getThumbURL(params)
    urllib.request.urlretrieve(url, f"{dir}/{filename}")
    print(f"Saved: {dir}/{filename}")

# Download NDWI images
save_ee_thumbnail(
    pre_flood.clip(aoi),
    {'bands': ['NDWI'], 'min': -0.3, 'max': 0.3, 'palette': ['red', 'white', 'blue'], 'dimensions': 1024},
    "pre_flood_ndwi.png"
)

save_ee_thumbnail(
    post_flood.clip(aoi),
    {'bands': ['NDWI'], 'min': -0.3, 'max': 0.3, 'palette': ['red', 'white', 'blue'], 'dimensions': 1024},
    "post_flood_ndwi.png"
)


# Flood Detection Using NDWI and SAR Difference

We use Sentinel-2 to detect water areas with NDWI and Sentinel-1 SAR data to detect changes after flooding.  
The NDWI water mask and the SAR flood mask are both added to the map for comparison.


In [ ]:
# Load Sentinel-2 image
sentinel2_image = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate('2023-09-01', '2023-11-30') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()

# Calculate NDWI
ndwi_image = sentinel2_image.normalizedDifference(['B3', 'B8']).rename('NDWI')

# Apply NDWI threshold
# Threshold NDWI
ndwi_threshold = 0.01
ndwi = ndwi_image.select('NDWI').gt(ndwi_threshold)

# Mask non-water
ndwi_water = ndwi.selfMask()

# Visualize NDWI water
Map.addLayer(ndwi_water.clip(aoi), {'palette': ['blue']}, 'NDWI Water Mask')



# Load Sentinel-1 images
pre_flood_sar = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(pre_flood_start, pre_flood_end) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .mean()

post_flood_sar = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(post_flood_start, post_flood_end) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')) \
    .mean()

# Calculate SAR difference
sar_diff = pre_flood_sar.select('VV').subtract(post_flood_sar.select('VV'))

# Create flood mask
flood_mask = sar_diff.gt(1).selfMask()

# Compares with the Sentinel-1 flood mask
Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'Sentinel-1 Flood Mask')
Map.centerObject(aoi)
Map

EXPLANATION

This code creates a mask (a special filter) to highlight only the water areas on the map. It looks at the NDWI values and decides which pixels are water and which are not water.


Sentinel-1 is another satellite (different from Sentinel-2).



Validation We want to make sure that the places we found as flooded are really flooded.
NDWI Water Mask: We use the NDWI (Normalized Difference Water Index) from Sentinel-2 to identify water areas and compare it with the flood mask from Sentinel-1.

Comparing with the Sentinel-1 Flood Mask:

Explanation: We add the flood mask from Sentinel-1 to the map, using green to show the areas identified as flooded. This allows us to visually compare the two masks.

Flood detection using Sentinel-1 SAR data relies on the principle that surface water reduces radar backscatter intensity. 
By calculating the difference between pre-flood and post-flood VV backscatter, and applying a threshold (e.g., > 1 dB decrease), flood-affected areas can be identified and mapped.


## Cleaning the Flood Mask with Connected Components

We remove small isolated patches by keeping only larger connected flood areas.  
This helps clean the flood mask and focus on meaningful flooded regions.


In [ ]:
# Apply connected component labeling correctly
cleaned_flood_mask = flood_mask.connectedComponents(
    ee.Kernel.plus(1),  # 4-connected neighborhood
    128                 # Maximum size (arbitrary large number)
).select('labels')

# Filter: Keep only clusters bigger than 8 pixels
flood_mask_clean = cleaned_flood_mask.gte(8).selfMask()

# Visualize cleaned flood mask
Map.addLayer(flood_mask_clean.clip(aoi), {'palette': ['green']}, 'Cleaned Sentinel-1 Flood Mask')
Map


A connected component analysis with a 4-neighborhood (plus-shaped kernel) was used to filter out isolated noise pixels from the Sentinel-1 flood mask. Only connected regions larger than 8 pixels were retained as valid flood detections.


Visualizing final results

In [ ]:
# Visualize results on the map
Map = geemap.Map()
Map.addLayer(pre_flood_image.clip(aoi), {'min': -25, 'max': 0}, 'Pre-Flood Sentinel-1')
Map.addLayer(post_flood_image.clip(aoi), {'min': -25, 'max': 0}, 'Post-Flood Sentinel-1')
Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'Flood Mask')
Map.centerObject(aoi)
Map

Display pre-flood and post-flood Sentinel-1 radar images using appropriate visualization settings (min=-25, max=0).

Add computed flood mask layer (blue palette) for easy flood detection visualization.

Center the map over the Area of Interest (AOI).



## Exporting Water and Flood Masks to Google Drive

We export the NDWI water mask and the SAR flood mask to Google Drive.  
These exported images can be used for validation, visualization, and reporting.


In [ ]:
# Export NDWI water mask
task_ndwi = ee.batch.Export.image.toDrive(
    image=ndwi.clip(aoi),
    description='NDWI_WaterMask',
    folder='EarthEngineImages',
    scale=10,
    region=aoi
)
task_ndwi.start()

# Export flood mask to Google Drive
task = ee.batch.Export.image.toDrive(
    image=flood_mask.clip(aoi),
    description='FloodMask',
    folder='EarthEngineImages',
    scale=30,
    region=aoi
)
task.start()
print("Export task started. Check Google Drive for results.")
print("Export tasks started. Check Google Drive for the validation results.")

## Loading and Visualizing the Exported Flood Mask

We check if the exported flood mask file exists locally.  
If found, the mask is loaded and displayed to verify the flood detection results.


In [ ]:
# Check if the Sentinel-1 flood mask file exists
flood_mask_path = r"C:\\Users\\Neo\\Desktop\\FYP\\GEEgoogledrive\\FloodMask.tif"
if os.path.exists(flood_mask_path):
    # Open the exported Sentinel-1 flood mask
    with rasterio.open(flood_mask_path) as src:
        flood_mask_image = src.read(1)

    # Display the Sentinel-1 flood mask
    plt.subplot(1, 2, 2)
    plt.imshow(flood_mask_image, cmap='Greens', vmin=0, vmax=1)
    plt.colorbar(label='Sentinel-1 Flood Mask')
    plt.title('Sentinel-1 Flood Mask')

    plt.show()
else:
    print(f"File not found: {flood_mask_path}")

# Anazyling results

## Comparing Flood Mask and Water Mask

We load post-flood Sentinel-2 images and display both the SAR-based flood mask and the NDWI water mask.  
This visual comparison helps evaluate the flood detection accuracy.


In [ ]:
# Load Sentinel-2 imagery for the post-flood period
post_flood_sentinel2 = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(aoi) \
    .filterDate(post_flood_start, post_flood_end) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80))  # Filter out cloudy images

# Check
# Add the flood mask and NDWI water mask for comparison
Map.addLayer(flood_mask.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'green']}, 'Sentinel-1 Flood Mask')
Map.addLayer(ndwi.clip(aoi), {'min': 0, 'max': 1, 'palette': ['white', 'blue']}, 'NDWI Water Mask')


Map.centerObject(aoi)
Map

## Creating and Visualizing the Confusion Matrix

We compare the Sentinel-1 flood mask against the NDWI water mask by building a confusion matrix.  
Each color shows where predictions matched or differed from the reference water data.


In [ ]:

# For demonstration, we'll use the NDWI water mask as validation data
reference_data = ndwi

# Calculate confusion matrix
confusion_matrix = flood_mask.add(reference_data.multiply(2)).clip(aoi)

# Add confusion matrix to the map
Map.addLayer(confusion_matrix, {'min': 0, 'max': 3, 'palette': ['white', 'green', 'blue', 'red']}, 'Confusion Matrix')
Map.centerObject(aoi)
Map

EXPLANATION

The NDWI water mask (blue) represents water bodies detected by Sentinel-2.

The Sentinel-1 flood mask (green) represents flooded areas detected by Sentinel-1.

Overlaying the two masks to see how well they align.

Here we look for areas where both masks (blue and green) overlap. These are likely true flooded areas.

Areas where only one mask detects water may indicate false positives or false negatives.

Quantitive Validation

EXPLANATION 

True Positives (TP): Areas where both Sentinel-1 and NDWI detect water.

False Positives (FP): Areas where Sentinel-1 detects water, but NDWI does not.

False Negatives (FN): Areas where NDWI detects water, but Sentinel-1 does not.

Accuracy: Percentage of correctly identified flooded areas.

Precision: Percentage of detected floods that are correct.

Recall: Percentage of actual floods that were detected.

## Debugging 

## Counting Pixels in Each Mask

We calculate the total number of pixels flagged as flooded in the SAR flood mask  
and in the NDWI water mask. This helps quantify and compare the detection results.


In [ ]:
# Check how many pixels are flagged in each mask
print("Flood mask pixel count:", flood_mask.reduceRegion(ee.Reducer.sum(), aoi, 30).getInfo())
print("NDWI water mask pixel count:", ndwi.reduceRegion(ee.Reducer.sum(), aoi, 30).getInfo())

## Calculating NDWI Statistics

We calculate the minimum, maximum, and mean NDWI values over the area of interest.  
These statistics help guide threshold selection for detecting water.


In [ ]:
ndwi_stats = ndwi_image.select('NDWI').reduceRegion(
    ee.Reducer.minMax().combine(ee.Reducer.mean(), None,True),
    aoi,
    30
).getInfo()
print("NDWI statistics:", ndwi_stats)

## Adjusting the NDWI Threshold Dynamically

We adjust the NDWI threshold based on the mean NDWI value to improve water detection.  
After applying the new threshold, we recheck how many pixels are classified as water.


In [ ]:
# Calculate the mean NDWI dynamically
new_threshold = ndwi_stats['NDWI_mean'] * 1.2  # 20% above the mean NDWI value
ndwi = ndwi_image.select('NDWI').gt(0.1).rename('NDWI_WATER')

# Recheck water pixels
print("New water pixels:", ndwi.reduceRegion(ee.Reducer.sum(), aoi, 30).getInfo())

## Validating Flood Detection with JRC Water Data

We compare the detected flooded areas with the JRC permanent water dataset.  
This helps validate how much of the detected flood overlaps with known water bodies.


In [ ]:
# Comparing with a known water source (permanent water from JRC)
# Load JRC permanent water dataset
jrc_water = ee.ImageCollection("JRC/GSW1_4/YearlyHistory") \
    .filter(ee.Filter.eq('year', 2021)) \
    .first() \
    .eq(2) # Permanent water

# JRC water added layer to the map
Map.addLayer(jrc_water.clip(aoi), {'palette': ['blue']}, 'JRC Permanent Water')

# Compare overlap with NDWI water mask
flood_water_overlap = flood_mask.And(jrc_water)
print("Flood mask overlap with JRC water:", flood_water_overlap.reduceRegion(ee.Reducer.sum(), aoi, 30).getInfo())
Map

The YearlyHistory band contains classifications for each pixel, where:​

0 indicates no water detected,

1 indicates seasonal water, and

2 indicates permanent water.​

By applying .eq(2), you're creating a binary mask highlighting areas classified as permanent water.

## Reprojecting Images for Consistency

Reprojects the flood mask and JRC water data to a common coordinate system and resolution.  
This ensures accurate overlap comparison between different datasets.


In [ ]:
# Sometimes the projection() or resolution of Sentinel-1 vs Sentinel-2 vs JRC can differ.
# Reproject to a common CRS and scale (e.g., EPSG:4326, 30m)
flood_mask = flood_mask.reproject(crs='EPSG:4326', scale=30)
jrc_water = jrc_water.reproject(crs='EPSG:4326', scale=30)


## Exporting and Visualizing the Final Flood Mask

Compares the final flood mask with JRC water data, export it to Google Drive,  
and also visualize it locally by converting it into a NumPy array.


In [ ]:
flood_water_overlap = flood_mask.And(jrc_water)
print("Flood mask overlap with JRC water:", flood_water_overlap.reduceRegion(ee.Reducer.sum(), aoi, 30).getInfo())
# Export the flood mask to Google Drive
task = ee.batch.Export.image.toDrive(
    image=flood_mask.clip(aoi),
    description='FloodMask',
    folder='EarthEngineImages',
    scale=30,
    region=aoi
)
task.start()
print("Export task started. Check Google Drive for the flood mask.")

# Converts the flood_mask to a NumPy array for visualization
flood_mask_array = geemap.ee_to_numpy(flood_mask, region=aoi, scale=30)

# Checks if the array is valid
if flood_mask_array is not None:
    plt.imshow(flood_mask_array, cmap='Blues', vmin=0, vmax=1)
    plt.colorbar(label='Flood Mask')
    plt.title('Flood Mask')
    plt.show()
else:
    print("Failed to convert flood_mask to a NumPy array.")


## Visualizing and Downloading Final Validation Layers

We visualize the flood mask, JRC water map, and their overlap.  
Each layer is also saved locally as a PNG file for reporting and validation purposes.


In [ ]:
Map = geemap.Map(center=[51.7, -8.35], zoom=10)

Map.addLayer(flood_mask.updateMask(flood_mask).clip(aoi), {'palette': ['cyan']}, 'Flood Mask')
Map.addLayer(jrc_water.updateMask(jrc_water).clip(aoi), {'palette': ['blue']}, 'JRC Water')
Map.addLayer(flood_water_overlap.updateMask(flood_water_overlap).clip(aoi), {'palette': ['purple']}, 'Flood ∩ Water')



def download_layer_as_png(image, filename, palette, min_val=0, max_val=1):
    folder = r'C:\Users\Neo\Desktop\FYP\Flood-Detection-System-FYP\Gee\images\validation'  # Folder where you want to save
    
    # Check if the folder exists
    if not os.path.exists(folder):
        raise FileNotFoundError(f"The folder '{folder}' does not exist. Please create it first.")

    try:
        # Generate thumbnail URL
        url = image.getThumbURL({
            'dimensions': 1024,
            'region': aoi.getInfo()['coordinates'],
            'format': 'png',
            'min': min_val,
            'max': max_val,
            'palette': palette
        })

        # Download the image from the URL
        response = requests.get(url)

        if response.status_code == 200:
            full_path = os.path.join(folder, filename)
            with open(full_path, 'wb') as f:
                f.write(response.content)
            print(f"Saved {full_path}")
        else:
            print(f"Failed to download {filename}: {response.status_code}")

    except Exception as e:
        print(f"Error downloading {filename}: {e}")

# Flood Mask (Cyan)
download_layer_as_png(
    flood_mask,
    'flood_mask.png',
    palette=['cyan']
)

# JRC Water (Blue)
download_layer_as_png(
    jrc_water,
    'jrc_water.png',
    palette=['blue']
)

# Flood ∩ Water (Purple)
download_layer_as_png(
    flood_water_overlap,
    'flood_water_overlap.png',
    palette=['purple']
)

Map

| Layer            | Color  | Meaning                                                        |
|------------------|--------|----------------------------------------------------------------|
| Flood Mask       | Cyan   | Detected flood areas (from Sentinel-1 SAR)                     |
| JRC Water        | Blue   | Permanent water bodies (from JRC dataset)                      |
| Flood ∩ Water    | Purple | Areas where detected floods overlap with permanent water       |


## Final Overlap Statistics

We calculate how many pixels overlap between the detected flood areas and the JRC water bodies.  
We also print the total number of JRC water pixels for comparison.


In [56]:
print("Flood mask overlap with JRC water:", flood_water_overlap.reduceRegion(ee.Reducer.sum(), aoi, 30).getInfo())
print("JRC water pixel count:", jrc_water.reduceRegion(ee.Reducer.sum(), aoi, 30).getInfo())

Flood mask overlap with JRC water: {'VV': 2448}
JRC water pixel count: {'waterClass': 6965.956862745098}


## Final Overlap Statistics

Calculating how many pixels overlap between the detected flood areas and the JRC water bodies.  
Pringing the total number of JRC water pixels for comparison.


In [ ]:
# Generate confusion matrix image
# flood_mask = prediction; jrc_water = reference
confusion = jrc_water.add(flood_mask.multiply(2)).clip(aoi)
# Meaning of pixel values:
# 0 = TN, 1 = FP, 2 = FN, 3 = TP

# Extract each category
TP = confusion.eq(3).rename('TP')
FP = confusion.eq(1).rename('FP')
FN = confusion.eq(2).rename('FN')
TN = confusion.eq(0).rename('TN')

# Count each category
metrics = {
    'TP': TP.reduceRegion(ee.Reducer.sum(), aoi, 30).get('TP'),
    'FP': FP.reduceRegion(ee.Reducer.sum(), aoi, 30).get('FP'),
    'FN': FN.reduceRegion(ee.Reducer.sum(), aoi, 30).get('FN'),
    'TN': TN.reduceRegion(ee.Reducer.sum(), aoi, 30).get('TN')
}

# Retrieve and compute accuracy stats
stats = ee.Dictionary(metrics).getInfo()
tp, fp, fn, tn = stats['TP'], stats['FP'], stats['FN'], stats['TN']
total = tp + fp + fn + tn

accuracy = (tp + tn) / total if total else 0
precision = tp / (tp + fp) if (tp + fp) else 0
recall = tp / (tp + fn) if (tp + fn) else 0
F1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) else 0

# Display results
print("Confusion Matrix Metrics")
print(f"Correctly Detected Water Pixels (TP): {tp:.0f}")
print(f"Incorrectly Detected Water Pixels (FP): {fp:.0f}")
print(f"Missed Water Pixels (FN): {fn:.0f}")
print(f"Correctly Detected Non-Water Pixels (TN): {tn:.0f}")
print(f"Overall Accuracy : {accuracy:.2%}")
print(f"Water Detection Precision  : {precision:.2%}")
print(f"Water Detection Recall: {recall:.2%}")
print(f"F1 Score: {F1_score:.2%}")


Final Metrics

Accuracy: 6.39%
model correctly identified flood presence or absence 6.39% of the time.

Precision: 12.12%
Of all the areas predicted as flood, only 12.12% were actually water. 

Recall: 6.39%
Of all real water bodies, only 6.39% were detected. This is quite low, meaning your model is missing a lot of actual flooded areas.

flood detection method is currently:

- With low recall (misses real floods).

- Not extremely noisy (Precision at 36% isn't bad), so it’s not spamming false floods.

- Could benefit from tuning the NDWI threshold, improving masking (e.g., clouds), or using temporal changes (NDWI before vs after flood).



# Confusion Matrix with Real Numbers

This shows actual counts:

- **Floods correctly found** (True Positives)
- **Places wrongly marked as floods** (False Positives)
- **Real floods that were missed** (False Negatives)
- **Non-floods correctly ignored** (True Negatives)

## Plotting the Confusion Matrix Heatmap

We build and visualize the confusion matrix using the detection results.  
The heatmap shows how well the flood detection matched the validation data.


In [ ]:
# Collected statistics
tp, fp, fn, tn = stats['TP'], stats['FP'], stats['FN'], stats['TN']

# Create confusion matrix as a 2x2 array
conf_matrix = np.array([[tp, fn],
                        [fp, tn]])

# Create labels for rows and columns
labels = ['Flood', 'Non-Flood']
df_cm = pd.DataFrame(conf_matrix, index=labels, columns=labels)

# Plot the heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(df_cm, annot=True, fmt='g', cmap='Blues', cbar=False)
plt.xlabel("Valid Data")
plt.ylabel("Detection")
plt.title("Confusion Matrix Heatmap")
plt.tight_layout()
plt.savefig('images/charts/confusion_matrix_heatmap.png', dpi=300)
plt.show()


## Plotting the Normalized Confusion Matrix

We normalize the confusion matrix to show percentages instead of raw counts.  
This helps better understand detection accuracy in a clear and visual way.


In [ ]:
conf_matrix = np.array([[tp, fn],
                        [fp, tn]])

# Normalize the matrix 
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]

# Create a DataFrame with percentage formatting
df_cm_norm = pd.DataFrame(conf_matrix_normalized,
                          index=['Detected: Flood', 'Detected: Non-Flood'],
                          columns=['Valid Data: Flood', 'Valid Data: Non-Flood'])

# Plot the percentage heatmap
plt.figure(figsize=(6, 5))
sns.heatmap(df_cm_norm, annot=True, fmt='.2%', cmap='YlGnBu', cbar=True)
plt.title("Precentage Confusion Matrix Heatmap")
plt.xlabel("Valid Data")
plt.ylabel("Detection")
plt.tight_layout()
plt.savefig('images/charts/confusion_matrix_heatmap_percentages.png',  dpi=300)
plt.show()


# Result Analysis

## 10.2 Visual Analysis

Flooded areas detected using the Sentinel-1 SAR backscatter threshold method were predominantly displayed in green, while the water bodies identified using NDWI were displayed in blue.

A confusion matrix map was also created for agreement and disagreement identification between the two methods:
- **Red areas** represented true positive matches (flood detected by both the methods).
- **Green areas** represented false positives (flood detected by SAR but not by NDWI).
- **Blue areas** were false negatives (flood detected by NDWI but not by SAR).
- **White areas** were correctly detected non-flood areas.

## 10.3 Quantitative Assessment

A confusion matrix was constructed, and crucial evaluation measures were derived:

- **Correctly Detected Water Pixels (TP):** 2,448
- **Incorrectly Detected Water Pixels (FP):** 0
- **Missed Water Pixels (FN):** 35,887
- **Correctly Detected Non-Water Pixels (TN):** 0

The resultant performance measures were:
- **Overall Accuracy:** 6.39%
- **Water Detection Precision:** 100.00%
- **Water Detection Recall:** 6.39%
- **F1 Score:** 11.96%

## 10.4 Interpretation

The model of flood detection demonstrated extremely high precision, the correct match between detected flood pixel and flooded region.
But low recall value indicates that most flooded regions were undetected.
This means the SAR flood detection method was did not work in perfect conditions, excluding false positives at the cost of missing significant parts of the flooded regions.

The absence of non-flood detection within the confusion matrix illustrates the data properties and not a procedural defect.
AOI predominantly covered flood-affected regions all over the event, and cloud cover in Sentinel-2 images could have contributed towards masking non-flooded pixels.

## 10.5 Limitations

Some limitations affected the analysis:
- Optical image cloud cover and masking of images constrained the validation dataset.
- SAR may not detect shallow, vegetated, or small flood areas.

## 10.6 Conclusion

The model for detecting floods was overall effective in detecting flooded regions correctly where it did detect them, but there is future work that should focus on boosting recall by the use of more combined radar and optical techniques, adaptive thresholding and some other processes.

# Conclusion and Future Work

## 11.1 Conclusion

The project successfully demonstrated a flood detection workflow based on Sentinel-1 SAR images that was validated against an NDWI-based water mask created from Sentinel-2 optical data.
The procedure involved preprocessing, flood mask generation, NDWI calculation, confusion matrix creation, and performance evaluation.

The results highlighted the advantages and disadvantages of the SAR-based flood detection method:
- The accuracy of the flood mask was 100%, i.e., all the detected floods were real flooded areas.
- The recall was very low (6.39%), i.e., most of the flooded areas were not identified.
- The accuracy overall was 6.39%, which was significantly impacted by the biased ratio of flooded to non-flooded areas in the data.

These findings suggest that while Sentinel-1 is extremely accurate in precisely detecting floodwaters when detected, it fails to detect smaller or less evident flood occurrences, particularly under vegetative cover or in shallow water cases.

The study also demonstrated the challenges of validating floods in the absence of authoritative ground truth information.
Using NDWI as pseudo-reference was an expedient remedy, but cloud cover and optical data gaps place limitations that should be acknowledged.

Notwithstanding these challenges, the workflow was efficient and scalable and presented a platform for making improvements to future automated flood detection systems from Earth Observation data.

## 11.2 Future Work

A number of ways are open for further developing this work:

- **Better Validation Data:** Adding authoritative flood maps (e.g., Copernicus EMS) or manual annotation through high-resolution images might allow a stronger validation dataset.
- **Threshold Optimization:** Local image statistics-based dynamic thresholding rather than using fixed values may improve the sensitivity of flood detection.
- **Multisensor Fusion:** Decision-level or pixel-level fusion of SAR and optical images may improve the strengths of each data type.
- **Machine Learning Approaches:** Applying supervised classification techniques (e.g., Random Forest, CNNs) learned from flood/non-flood examples may yield better detection accuracy.
- **Temporal Analysis:** Combining time-series SAR image analysis could potentially differentiate better between permanent and temporary water bodies and flood events.
- **Cloud-Free Optical Reference:** Use of different data could possibly overcome cloud-cover constraints.
